In [25]:
import pandas as pd
from parse import Parse

from dbconnection import DBConnection
import configparser
from helper_function import handle_exception
import numpy as np


config = configparser.ConfigParser()
config.read('config.ini')

conn_param = 'DATA'
borrower_table_url = config[conn_param]['BORROWER_URL']
loan_data_url = config[conn_param]['LOAN_DATA_URL']
payment_schedule_url = config[conn_param]['PAYMENT_SCHEDULE_URL']
repayment_data_url = config[conn_param]['REPAYMENT_DATA_URL']


create_table_script = config[conn_param]['CREATE_TABLE_SCRIPT']
result_script = config[conn_param]['RESULT_SCRIPT']


parse = Parse()
db = DBConnection(parse)

try:
    borrower_df = pd.read_csv(parse.parse_url(borrower_table_url))
    loan_df = pd.read_csv(parse.parse_url(loan_data_url))
    payment_schedule_df = pd.read_csv(parse.parse_url(payment_schedule_url))
    repayment_data_df = pd.read_csv(parse.parse_url(repayment_data_url))

    
except Exception as err:
    handle_exception(err)


In [26]:
borrower_df = parse.change_column_dtype(borrower_df)
loan_df = parse.change_column_dtype(loan_df)
payment_schedule_df = parse.change_column_dtype(payment_schedule_df)
repayment_data_df = parse.change_column_dtype(repayment_data_df)

31 1 2022
29 2 2023


In [28]:
repayment_data_df.head()

,loan_id(fk),payment_id(pk),Amount_paid,Date_paid
0,32u09wekjbfje,3434r409kmPAID123456,2021-03-31,100790.3333
1,32u09wekjbfje,3434r409kmPAID123457,2021-03-31,100790.3333
2,32u09wekjbfje,3434r409kmPAID123458,2021-04-27,100790.3333
3,32u09wekjbfje,3434r409kmPAID123459,2021-05-27,100790.3333
4,32u09wekjbfje,3434r409kmPAID123460,2021-06-27,100790.3333


### Generate schedule id from payment_id(pk). Column will be used to merge with repayment data

In [29]:


repayment_data_df['schedule_id'] = repayment_data_df['payment_id(pk)'].apply(lambda x: x.replace('PAID',''))

In [33]:
# merge  payment and repayment table schedule_id and loan_id

merge = pd.merge(repayment_data_df,payment_schedule_df,how='left',left_on=['loan_id(fk)','schedule_id'],right_on=['loan_id','schedule_id'])

# merge  (payment,repayment) and loan table and loan_id

merge = pd.merge(merge,loan_df,how='left',left_on='loan_id(fk)',right_on='loan_id')

# merge  (payment,repayment,loan) and borrower table on Borrower_id

merge = pd.merge(merge,borrower_df,how='left',left_on='Borrower_id',right_on='Borrower_Id')



In [34]:
merge.head()

,loan_id(fk),payment_id(pk),Amount_paid,Date_paid,schedule_id,loan_id_x,Expected_payment_date,Expected_payment_amount,Borrower_id,loan_id_y,...,InterestRate,LoanAmount,Downpayment,Payment_frequency,Maturity_date,Borrower_Id,State,City,zip code,borrower_credit_score
0,32u09wekjbfje,3434r409kmPAID123456,2021-03-31,100790.3333,3434r409km123456,32u09wekjbfje,2021-02-27,100790.3333,123fd35,32u09wekjbfje,...,1.05,1209484,124993,100790.3333,01/31/2022,123fd35,dfgc,12olki,19473,4
1,32u09wekjbfje,3434r409kmPAID123457,2021-03-31,100790.3333,3434r409km123457,32u09wekjbfje,2021-03-27,100790.3333,123fd35,32u09wekjbfje,...,1.05,1209484,124993,100790.3333,01/31/2022,123fd35,dfgc,12olki,19473,4
2,32u09wekjbfje,3434r409kmPAID123458,2021-04-27,100790.3333,3434r409km123458,32u09wekjbfje,2021-04-27,100790.3333,123fd35,32u09wekjbfje,...,1.05,1209484,124993,100790.3333,01/31/2022,123fd35,dfgc,12olki,19473,4
3,32u09wekjbfje,3434r409kmPAID123459,2021-05-27,100790.3333,3434r409km123459,32u09wekjbfje,2021-05-27,100790.3333,123fd35,32u09wekjbfje,...,1.05,1209484,124993,100790.3333,01/31/2022,123fd35,dfgc,12olki,19473,4
4,32u09wekjbfje,3434r409kmPAID123460,2021-06-27,100790.3333,3434r409km123460,32u09wekjbfje,2021-06-27,100790.3333,123fd35,32u09wekjbfje,...,1.05,1209484,124993,100790.3333,01/31/2022,123fd35,dfgc,12olki,19473,4


In [35]:
merge.rename(columns={'Amount_paid':'date_paid'},inplace=True)
merge.rename(columns={'Date_paid':'Amount_paid'},inplace=True)

In [36]:
merge.head()

,loan_id(fk),payment_id(pk),date_paid,Amount_paid,schedule_id,loan_id_x,Expected_payment_date,Expected_payment_amount,Borrower_id,loan_id_y,...,InterestRate,LoanAmount,Downpayment,Payment_frequency,Maturity_date,Borrower_Id,State,City,zip code,borrower_credit_score
0,32u09wekjbfje,3434r409kmPAID123456,2021-03-31,100790.3333,3434r409km123456,32u09wekjbfje,2021-02-27,100790.3333,123fd35,32u09wekjbfje,...,1.05,1209484,124993,100790.3333,01/31/2022,123fd35,dfgc,12olki,19473,4
1,32u09wekjbfje,3434r409kmPAID123457,2021-03-31,100790.3333,3434r409km123457,32u09wekjbfje,2021-03-27,100790.3333,123fd35,32u09wekjbfje,...,1.05,1209484,124993,100790.3333,01/31/2022,123fd35,dfgc,12olki,19473,4
2,32u09wekjbfje,3434r409kmPAID123458,2021-04-27,100790.3333,3434r409km123458,32u09wekjbfje,2021-04-27,100790.3333,123fd35,32u09wekjbfje,...,1.05,1209484,124993,100790.3333,01/31/2022,123fd35,dfgc,12olki,19473,4
3,32u09wekjbfje,3434r409kmPAID123459,2021-05-27,100790.3333,3434r409km123459,32u09wekjbfje,2021-05-27,100790.3333,123fd35,32u09wekjbfje,...,1.05,1209484,124993,100790.3333,01/31/2022,123fd35,dfgc,12olki,19473,4
4,32u09wekjbfje,3434r409kmPAID123460,2021-06-27,100790.3333,3434r409km123460,32u09wekjbfje,2021-06-27,100790.3333,123fd35,32u09wekjbfje,...,1.05,1209484,124993,100790.3333,01/31/2022,123fd35,dfgc,12olki,19473,4


In [37]:
merge.dtypes

loan_id(fk)                        object
payment_id(pk)                     object
date_paid                  datetime64[ns]
Amount_paid                       float64
schedule_id                        object
loan_id_x                          object
Expected_payment_date      datetime64[ns]
Expected_payment_amount           float64
Borrower_id                        object
loan_id_y                          object
Date_of_release            datetime64[ns]
Term                                int64
InterestRate                      float64
LoanAmount                          int64
Downpayment                         int64
Payment_frequency                 float64
Maturity_date                      object
Borrower_Id                        object
State                              object
City                               object
zip code                            int64
borrower_credit_score              object
dtype: object

In [40]:
# Calculating amount risk based on condition stated

merge['risk'] = np.where(merge['date_paid']>merge['Expected_payment_date'],merge['Expected_payment_amount'],0)

In [41]:
# Calculating current days past expected days
merge['Current_days_past_due'] = merge['date_paid'] - merge['Expected_payment_date']

In [42]:
merge.head()

,loan_id(fk),payment_id(pk),date_paid,Amount_paid,schedule_id,loan_id_x,Expected_payment_date,Expected_payment_amount,Borrower_id,loan_id_y,...,Downpayment,Payment_frequency,Maturity_date,Borrower_Id,State,City,zip code,borrower_credit_score,risk,Current_days_past_due
0,32u09wekjbfje,3434r409kmPAID123456,2021-03-31,100790.3333,3434r409km123456,32u09wekjbfje,2021-02-27,100790.3333,123fd35,32u09wekjbfje,...,124993,100790.3333,01/31/2022,123fd35,dfgc,12olki,19473,4,100790.3333,32 days
1,32u09wekjbfje,3434r409kmPAID123457,2021-03-31,100790.3333,3434r409km123457,32u09wekjbfje,2021-03-27,100790.3333,123fd35,32u09wekjbfje,...,124993,100790.3333,01/31/2022,123fd35,dfgc,12olki,19473,4,100790.3333,4 days
2,32u09wekjbfje,3434r409kmPAID123458,2021-04-27,100790.3333,3434r409km123458,32u09wekjbfje,2021-04-27,100790.3333,123fd35,32u09wekjbfje,...,124993,100790.3333,01/31/2022,123fd35,dfgc,12olki,19473,4,0.0000,0 days
3,32u09wekjbfje,3434r409kmPAID123459,2021-05-27,100790.3333,3434r409km123459,32u09wekjbfje,2021-05-27,100790.3333,123fd35,32u09wekjbfje,...,124993,100790.3333,01/31/2022,123fd35,dfgc,12olki,19473,4,0.0000,0 days
4,32u09wekjbfje,3434r409kmPAID123460,2021-06-27,100790.3333,3434r409km123460,32u09wekjbfje,2021-06-27,100790.3333,123fd35,32u09wekjbfje,...,124993,100790.3333,01/31/2022,123fd35,dfgc,12olki,19473,4,0.0000,0 days


In [43]:
#  Calculating Amount at risk
merge['Amount_at_risk'] = merge.sort_values('Expected_payment_date').groupby(['loan_id(fk)','Borrower_id'])['risk'].cumsum()

In [44]:
# Calculating Total Amount Expected
merge['Total_amount_expected'] = merge.sort_values('Expected_payment_date').groupby(['loan_id(fk)','Borrower_id'])['Expected_payment_amount'].cumsum()

In [46]:
# Calculating Total Amount Expected
merge['Total_amount_paid'] = merge.sort_values('date_paid').groupby(['loan_id(fk)','Borrower_id'])['Expected_payment_amount'].cumsum()

In [49]:
# Generating None values for additional columns. Values was not specified.
merge['branch'] = None
merge['branch_id'] = None
merge['borrower_name'] = None

In [57]:
# Rename columns in the merged table and extract out the needed columns
rename_column = {
                    'loan_id(fk)': 'loan_id',
                    'Borrower_id': 'borrower_id',
                    'Date_of_release': 'loan_date_of_release',
                    'Term': 'term',
                    'LoanAmount': 'LoanAmount',
                    'Downpayment': 'Downpayment',
                    'State': 'state',
                    'City': 'city',
                    'zip code': 'zip code',
                    'Payment_frequency': 'payment_frequency',
                    'Maturity_date': 'maturity_date',
                    'Current_days_past_due': 'current_days_past_due',
                    'Expected_payment_date': 'last_due_date',
                    'date_paid': 'last_repayment_date',
                    'Amount_at_risk': 'amount_at_risk',
                    'borrower_credit_score': 'borrower_credit_score',
                    'branch': 'branch',
                    'branch_id': 'branch_id',
                    'borrower_name': 'borrower_name',
                    'Total_amount_paid': 'total_amount_paid',
                    'Total_amount_expected': 'total_amount_expected'  
                }
merge.rename(columns=rename_column,inplace=True)
merge[[v for v in rename_column.values()]].to_csv('output2.csv')

In [ ]:
merge[[]]